In [2]:
import nltk
from nltk import load_parser
from nltk.sem import chat80

Your Turn: Run the parser with maximum tracing on, i.e., cp =
load_parser('grammars/book_grammars/sql0.fcfg', trace=3), and examine
how the values of SEM are built up as complete edges are added
to the chart.

In [7]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg',trace=3)
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

|.W.c.a.l.i.C.|
Leaf Init Rule:
|[-] . . . . .| [0:1] 'What'
|. [-] . . . .| [1:2] 'cities'
|. . [-] . . .| [2:3] 'are'
|. . . [-] . .| [3:4] 'located'
|. . . . [-] .| [4:5] 'in'
|. . . . . [-]| [5:6] 'China'
Feature Bottom Up Predict Combine Rule:
|[-] . . . . .| [0:1] Det[SEM='SELECT'] -> 'What' *
Feature Bottom Up Predict Combine Rule:
|[-> . . . . .| [0:1] NP[SEM=(?det+?n)] -> Det[SEM=?det] * N[SEM=?n] {?det: 'SELECT'}
Feature Bottom Up Predict Combine Rule:
|. [-] . . . .| [1:2] N[SEM='City FROM city_table'] -> 'cities' *
Feature Single Edge Fundamental Rule:
|[---] . . . .| [0:2] NP[SEM=(SELECT, City FROM city_table)] -> Det[SEM='SELECT'] N[SEM='City FROM city_table'] *
Feature Bottom Up Predict Combine Rule:
|[---> . . . .| [0:2] S[SEM=(?np+WHERE+?vp)] -> NP[SEM=?np] * VP[SEM=?vp] {?np: (SELECT, City FROM city_table)}
Feature Bottom Up Predict Combine Rule:
|. . [-] . . .| [2:3] IV[SEM=''] -> 'are' *
Feature Bottom Up Predict Combine Rule:
|. . [-> . . .| [2:3] VP[SEM=(?v+?pp)] 

Your Turn: Extend the grammar sql0.fcfg so that it will translate (4a) into (4b), and check the values returned by the query.
a.		What cities are in China and have populations above 1,000,000?

b.		SELECT City FROM city_table WHERE Country = 'china' AND Population > 1000
You will probably find it easiest to first extend the grammar to handle queries like What cities have populations above 1,000,000 before tackling conjunction. After you have had a go at this task, you can compare your solution to grammars/book_grammars/sql1.fcfg in the NLTK data distribution.

In [ ]:
ch_10 = load_parser('file:sql0.fcfg')
query = 'What cities have populations above 1,000,000'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

1. Translate the following sentences into propositional logic and verify that they can be processed with Expression.fromstring(). Provide a key which shows how the propositional variables in your translation correspond to expressions of English.

If Angus sings, it is not the case that Bertie sulks.
Cyril runs and barks.
It will snow if it doesn't rain.
It's not the case that Irene will be happy if Olive or Tofu comes.
Pat didn't cough or sneeze.
If you don't come if I call, I won't come if you call.

In [1]:
import nltk, re, pprint

In [2]:
read_expr = nltk.sem.Expression.fromstring
read_expr('P -> Q')
read_expr('P & Q')
read_expr('-P -> Q')
read_expr('-P -> Q')
read_expr('-(P | Q)')
read_expr('(P -> -Q) -> (Q -> -P)')

<ImpExpression ((P -> -Q) -> (Q -> -P))>

In [3]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('o', 'c'), ('c', 'b'), ('b', 'o')},
 'walk': {('c',), ('o',)}}


In [7]:
dom = {'b', 'o', 'c'}
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
m = nltk.Model(dom, val)
m.evaluate('exists x.(girl(x) & walk(x))', g)

True

In [12]:
read_expr = nltk.sem.Expression.fromstring
fmla1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fmla1, 'x', g)

{'b', 'o'}

In [13]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)


{'b', 'c', 'o'}

In [14]:

fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

{'b', 'o'}

In [20]:
m.evaluate('all x.(girl(x) & walk(x))', g)

False

In [16]:
m.evaluate('exists x.(boy(x) -> walk(x))', g)

True

In [21]:

>>> v2 = """
... bruce => b
... elspeth => e
... julia => j
... matthew => m
... person => {b, e, j, m}
... admire => {(j, b), (b, b), (m, e), (e, m)}
... """
>>> val2 = nltk.Valuation.fromstring(v2)

In [22]:
>>> dom2 = val2.domain
>>> m2 = nltk.Model(dom2, val2)
>>> g2 = nltk.Assignment(dom2)
>>> fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
>>> m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

2. Translate the following sentences into predicate-argument formulas of first-order logic.
a. Angus likes Cyril and Irene hates Cyril.
b. Tofu is taller than Bertie.
c. Bruce loves himself and Pat does too.
d. Cyril saw Bertie, but Angus didn’t.
e. Cyril is a four-legged friend.
f. Tofu and Olive are near each other.

'like(a, c) & hate(i,c)'

'TallerThan(t, b)'

'Love(b,b) & love(p,b)'

'see(c,b) & -see(a,b)'

'fourLegs(c)'

'Near(t,o) & Near(o,t)'

Translate the following sentences into quantified formulas of first-order logic.
a. Angus likes someone and someone likes Julia.
b. Angus loves a dog who loves him.
c. Nobody smiles at Pat.
d. Somebody coughs and sneezes.
e. Nobody coughed or sneezed.
f. Bruce loves somebody other than Bruce.
g. Nobody other than Matthew loves Pat.
h. Cyril likes everyone except for Irene.
i. Exactly one person is asleep.

'exists x.like(a,x) & exists y.like(y,j)'

'exists x.(dog(x) & love(a,x) & love(x,a))'

'-(exists x.smiles(x,p))'

'exists x.(cough(x) & sneeze(x))'

'-(exists x.(cough(x) | sneeze(x))'

? 'exist x.love(b,x), b!=x'


☼ Translate the following verb phrases using λ abstracts. quantified formulas of first order logic.

feed Cyril and give a capuccino to Angus
be given 'War and Peace' by Pat
be loved by everyone
be loved or detested by everyone
be loved by everyone and detested by no-one


\x.(feed(x,cyril) & give(x,cappuccino,angus))
\x.(give(pat,'War and Peace',x))
all x.(\x.love(y,x))
all x.(\x.love(y,x) | detest(y,x))
all x.(\x.love(y,x) & -detest(y,x))

In [ ]:
#5
>>> read_expr = nltk.sem.Expression.fromstring
>>> e2 = read_expr('pat')
>>> e3 = nltk.sem.ApplicationExpression(e1, e2)
>>> print(e3.simplify())
exists y.love(pat, y)

Clearly something is missing here, namely a declaration of the value of e1. In order for ApplicationExpression(e1, e2) to be β-convertible to exists y.love(pat, y), e1 must be a λ-abstract which can take pat as an argument. Your task is to construct such an abstract, bind it to e1, and satisfy yourself that the statements above are all satisfied (up to alphabetic variance). In addition, provide an informal English translation of e3.simplify().

Now carry on doing this same task for the further cases of e3.simplify() shown below.

In [ ]:
>>> print(e3.simplify())
exists y.(love(pat,y) | love(y,pat))

>>> print(e3.simplify())
exists y.(love(pat,y) | love(y,pat))

>>> print(e3.simplify())
walk(fido)